In [ ]:
I_GPU = 0

In [ ]:
import torch
import MinkowskiEngine as ME
import torchsparse as TS
torch.cuda.set_device(I_GPU)

In [ ]:
lib = TS
requires_grad = True


n = 20
dimension = 3
in_feat_torch = 5
in_feat_sparse = 2
out_feat_sparse = 1
n_batch = 1

In [ ]:
# Create coords and preprend batch id in first column
coords = torch.randint(low=0, high=5, size=(n, dimension)).int()

print("WARNING : MUST QUANTIZE BEFORE TORCHSPARSE SPARSETENSOR INSTANTIATION  (use ME quantization or GridSampling3d")
# You can use torchsparse.utils.sparse_quantize but it has a bug so need to edit torchsparse source locally...
# this is just for experimenting here, be careful because unique() changes the index order 
coords = torch.unique(coords, dim=0)  # this is for torchsparse, which does not handle duplicates in input...

batch = torch.randint(low=0, high=n_batch, size=(coords.shape[0], 1)).int()

# Create features of the size of coords
x = torch.rand(batch.shape[0], in_feat_torch, requires_grad=requires_grad).float()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Pass X in a torch operation
torch_conv = torch.nn.Linear(in_features=in_feat_torch, out_features=in_feat_sparse, bias=True)
XA = torch_conv(x)

if lib == ME:
	batch_coords = torch.cat((batch, coords), 1)  # in ME batch is the FIRST column  (torch_points3d/modules/SparseConv3d/nn/minkowski.py)
else:
	batch_coords = torch.cat((coords, batch), 1)  # in TS batch is the LAST column  (torch_points3d/modules/SparseConv3d/nn/torchsparse.py)
    
# Create the sparse tensors
if lib == ME:
	A = lib.SparseTensor(XA, batch_coords, quantization_mode=lib.SparseTensorQuantizationMode.UNWEIGHTED_AVERAGE).to(device)
else:
	A = lib.SparseTensor(XA, batch_coords).to(device)
	A.check()  # initialize the coord_maps
# print("Sparse tensor A")
# print(f"Num points : {A.C.shape[0]}")
# print("Sparse coordinates\n", A.C)
# print("Features\n", A.F)
# print()

XB = torch.rand(batch_coords.shape[0], in_feat_sparse, requires_grad=requires_grad).float()
if lib == ME:
	B = lib.SparseTensor(XB, batch_coords, quantization_mode=lib.SparseTensorQuantizationMode.UNWEIGHTED_AVERAGE).to(device)
	# B._manager = A.coordinate_manager  # ME v0.5
	B.coords_man = A.coords_man  # ME v0.4
else:
	B = lib.SparseTensor(XB, batch_coords).to(device)
	B.check()  # initialize the coord_maps
# print("Sparse tensor B")
# print(f"Num points : {B.C.shape[0]}")
# print("Sparse coordinates\n", B.C)
# print("Features\n", B.F)
# print()


# Sparse operations on the tensors
C = A + B  # WARNING : SparseTensor s are assumed to have the same coords !
# print("A + B            (assuming identical coordinates)")
# print(f"Num points : {C.C.shape[0]}")
# print("Sparse coordinates\n", C.C)
# print("Features\n", C.F)
# print()


if lib == ME:
	C = lib.cat(A, C)    # WARNING : SparseTensor s are assumed to have the same coords !
else:	
	C = lib.cat([A, C])  # WARNING : SparseTensor s are assumed to have the same coords !
# print("cat(A, C)        (assuming identical coordinates)")
# print(f"Num points : {C.C.shape[0]}")
# print("Sparse coordinates\n", C.C)
# print("Features\n", C.F)
# print()


# Strided conv
if lib == ME:
	sparse_conv_1 = ME.MinkowskiConvolution(in_channels=C.F.shape[1], out_channels=out_feat_sparse, kernel_size=3, stride=2, dimension=dimension).to(device)
else:
	import torchsparse.nn as spnn
	sparse_conv_1 = spnn.Conv3d(in_channels=C.F.shape[1], out_channels=out_feat_sparse, kernel_size=3, stride=2, dilation=1).to(device)
C = sparse_conv_1(C)
# print("sparse_conv_1(C)")
# print(f"Num points : {C.C.shape[0]}")
# print("Sparse coordinates\n", C.C)
# print("Features\n", C.F)
# print()


# Non-strided conv
if lib == ME:
	sparse_conv_2 = ME.MinkowskiConvolution(in_channels=C.F.shape[1], out_channels=out_feat_sparse, kernel_size=3, stride=1, dimension=dimension).to(device)
else:
	import torchsparse.nn as spnn
	sparse_conv_2 = spnn.Conv3d(in_channels=C.F.shape[1], out_channels=out_feat_sparse, kernel_size=3, stride=1, dilation=1).to(device)
D = sparse_conv_2(C)
# print("sparse_conv_2(C)")
# print(f"Num points : {D.C.shape[0]}")
# print("Sparse coordinates\n", D.C)
# print("Features\n", C.F)
# print()


# Strided conv
if lib == ME:
	sparse_conv_2 = ME.MinkowskiConvolution(in_channels=D.F.shape[1], out_channels=out_feat_sparse, kernel_size=3, stride=2, dimension=dimension).to(device)
else:
	import torchsparse.nn as spnn
	sparse_conv_2 = spnn.Conv3d(in_channels=D.F.shape[1], out_channels=out_feat_sparse, kernel_size=3, stride=2, dilation=1).to(device)
E = sparse_conv_2(D)
# print("sparse_conv_2(C)")
# print(f"Num points : {E.C.shape[0]}")
# print("Sparse coordinates\n", E.C)
# print("Features\n", C.F)
# print()


Y = E.F.sum()


# print("Before backward")
# print("[Is None] Input x grad : ", x.grad is None)
# print("[Is None] Sparse conv kernel grad : ", list(sparse_conv_1.parameters())[0].grad is None)
# print()


# print("Y.backward    (only works on GPU if using torchsparse)")
# print()
# Y.backward()  # will cause troubles for torchsparse on the CPU. GPU should be OK otherwise 


# print("After backward")
# print("[Is None] Input x grad : ", x.grad is None)
# print("[Is None] Sparse conv kernel grad : ", list(sparse_conv_1.parameters())[0].grad is None)
# print()
# print("Input x grad\n", x.grad)

In [ ]:
if lib == ME:
    # Using inverse mappings for input sparse nodes
    # Allows restoring the mapping from quantized to input quantized in case there
    # are duplicates in the input coords
    print("Inverse_mapping maps to input quantized coords : ", torch.all(A.C[A.inverse_mapping] == batch_coords))  # ME v0.5

    print("Unique index tells you which coords were taken in the input :", A.unique_index)


    # Input quantization ME v0.5
    unique_map, inverse_map = ME.utils.quantization.quantize(batch_coords)
    print("Quantization of input coords yields unique_map==A.unique_index and inverse_map")
    print(torch.all(A.unique_index == unique_map))


    # coord_key : which sparse coordinate system resolution each tensor belongs to
    print('Coords keys')
    for st in [A, B, C, D, E]:
        print(st.coords_key)
    print()

    print('Tensor stride')
    for st in [A, B, C, D, E]:
        print(st.tensor_stride)
    print()

    print('Tensor dimension')
    for st in [A, B, C, D, E]:
        print(st.D)
    print()

    print('Tensor feature shape')
    for st in [A, B, C, D, E]:
        print(st.size())
    print()


# Mappings
if lib == ME:
    print('Mapping from stride i to stride j')
    cm = E.coords_man
    src, target = cm.get_coords_map(1, 4)
    print(f"Source idx: {src}")
    print(f"Target idx: {target}")
    print(f"Source-sorted target idx: {target[src.argsort()]}")

if lib == TS :
    from torchsparse.nn.functional import sphash, sphashquery
    
    strides = list(E.coord_maps.keys())
    print(f"Strides (resolutions) in the network : {strides}")
    
    print('Mapping from stride i to stride j')
    ratio = strides[-1] / strides[0]
    in_coords = torch.floor(torch.floor(E.coord_maps[strides[0]].float() / ratio) * ratio).int()
    out_coords = E.coord_maps[strides[-1]]
    print(f"in_coords: \n{in_coords}")
    print(f"D.coords")
    print(f"out_coords: \n{out_coords}")
    print(sphashquery(sphash(in_coords), sphash(out_coords)))

In [ ]:
a = torch.Tensor([2, 2, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 4])
b = torch.Tensor([1, 1, 2, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 3])

for i in torch.unique(a):
    idx = torch.where(a == i)[0]
    print((b[idx] == (b[idx[0]] * torch.ones(idx.shape[0]))).all())